In [1]:
import sys
sys.path.append('Covid_Classify/code')
import pandas as pd
import util
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import classifynet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score  # Import cross_val_score
from xgboost import XGBClassifier, plot_importance
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import librosa
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv('metadata_compiled.csv')
df_filtered = df[['uuid','status']]
df_filtered = df_filtered.dropna()

In [3]:
x = []
y = []
features,label = util.filterdf(df_filtered,'symptomatic','Audio')
x += features
y += label
features,label = util.filterdf(df_filtered,'symptomatic','Audio')
x += features
y += label
features,label = util.filterdf(df_filtered,'healthy','Audio')
x += features
y += label

In [61]:
from imblearn.over_sampling import SMOTE

smote=SMOTE()
x, y = smote.fit_resample(x, y)
# Normalize the features using StandardScaler
scaler = StandardScaler() 
x = scaler.fit_transform(x)

In [ ]:
X_arr = np.array(x)  # Convert to numpy array
Y_arr = np.array(y)  # Convert to numpy array

# Check if X and y are populated correctly
print(f"Number of feature vectors: {len(X_arr)}")
print(f"Number of labels: {len(Y_arr)}")


if len(X_arr) > 0:
    print(f"Sample features: {X_arr[0]}")
    print(f"Sample label: {Y_arr[0]}")

In [63]:
# Label encode the genre labels
label_encoder = LabelEncoder()
Y_arr = label_encoder.fit_transform(Y_arr)  # Converts genre names to numeric labels
#y_test_encoded = label_encoder.transform(Y_test)
X_train,X_test,Y_train,Y_test = train_test_split(X_arr, Y_arr, test_size=0.2, random_state=0)


In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='auc')

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, Y_train, cv=5)  # Using 5-fold cross-validation
print(f'Cross-Validation Scores: {cv_scores}')
print(f'Mean Cross-Validation Score: {np.mean(cv_scores):.2f}')

# Fit the model on the full training set after cross-validation
model.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Decode predictions back to genre names
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(Y_test)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Print classification report
print(classification_report(Y_test, y_pred, target_names=label_encoder.classes_))

# Confusion Matrix Visualization
conf_matrix = confusion_matrix(Y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix: Actual vs Predicted')
plt.show()

# Plot Feature Importance
plt.figure(figsize=(12, 10))
plot_importance(model, max_num_features=10, importance_type='weight')  # Adjust max_num_features as needed
plt.title('Feature Importance')
plt.show()

In [ ]:
X_new = util.audio_vec('fffce9f0-a5e8-4bee-b13b-c671aac4a61c.webm','Audio',train=0)
print(X_new)
scaler = StandardScaler() 
X_train_ = scaler.fit_transform(X_new.reshape(1,-1))
y_pred = model.predict(X_train_)
print(y_pred)
# Decode predictions back to genre names
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_pred_decoded

In [12]:
model.save_model('covid_classify.json')
np.save('classes.npy', label_encoder.classes_)